In [1]:
from google.colab import files
import io
import pandas as pd

uploaded = files.upload()

filename = next(iter(uploaded))

if filename.lower().endswith((".xlsx", ".xls")):
    data = pd.read_excel(io.BytesIO(uploaded[filename]), sheet_name=0, engine="openpyxl")
else:
    data = pd.read_csv(io.BytesIO(uploaded[filename]))

print("Loaded:", filename, data.shape)
data.head()

Saving HSI_HSIF0923_calendar_filled.xlsx to HSI_HSIF0923_calendar_filled.xlsx
Loaded: HSI_HSIF0923_calendar_filled.xlsx (38, 4)


,Date,HSI,HSIF0923,IsTradingDay
0,2023-08-01,20011,20035,True
1,2023-08-02,19517,19537,True
2,2023-08-03,19421,19440,True
3,2023-08-04,19539,19523,True
4,2023-08-05,19539,19523,False


In [2]:
import math
import pandas as pd

N_CONTRACTS = 8
IM, MM = 101_944.0, 81_555.0
R = 0.02
MULTIPLIER = 50.0                       # HSI futures multiplier
COL_FUT = "HSIF0923"

data = data.sort_values("Date").reset_index(drop=True)
data["Date"] = pd.to_datetime(data["Date"])
data["dF"] = data[COL_FUT].diff().fillna(0.0)

# No fee in Q1 but may used in Q2
data["Fee"] = 0.0

#  iterate: interest + MTM + margin calls
init_total  = N_CONTRACTS * IM
maint_total = N_CONTRACTS * MM
daily_factor = math.exp(R / 365.0)

balances, deposits, interests, vms = [], [], [], []

B_prev = init_total
balances.append(B_prev); deposits.append(0.0); interests.append(0.0); vms.append(0.0)

for i in range(1, len(data)):
    interest = B_prev * (daily_factor - 1.0)                  # daily interest (weekends included)
    vm = N_CONTRACTS * MULTIPLIER * data.loc[i, "dF"]         # MTM vs previous calendar day
    B_tmp = B_prev + interest + vm  # no fee subtraction here

    deposit = 0.0
    if B_tmp < maint_total:                                   # margin call -> top up to init_total
        deposit = init_total - B_tmp
        B_tmp += deposit

    interests.append(interest); vms.append(vm); deposits.append(deposit); balances.append(B_tmp)
    B_prev = B_tmp

# output
q1_table = pd.DataFrame({
    "Date": data["Date"].dt.date,
    "FuturesPrice": data[COL_FUT],
    "dF": data["dF"],
    "VM": vms,
    "Interest": interests,
    "Fee": data["Fee"],         # stays 0 for all rows in Q1
    "Deposit": deposits,
    "Balance": balances
})
q1_table[["dF","VM","Interest","Fee","Deposit","Balance"]] = q1_table[["dF","VM","Interest","Fee","Deposit","Balance"]].round(2)

q1_table.to_csv("Q1_margin_table.csv", index=False, encoding="utf-8-sig")
print("Saved: Q1_margin_table.csv   Shape:", q1_table.shape)
q1_table


Saved: Q1_margin_table.csv   Shape: (38, 8)


,Date,FuturesPrice,dF,VM,Interest,Fee,Deposit,Balance
0,2023-08-01,20035,0.0,0.0,0.00,0.0,0.00,815552.00
1,2023-08-02,19537,-498.0,-199200.0,44.69,0.0,199155.31,815552.00
2,2023-08-03,19440,-97.0,-38800.0,44.69,0.0,0.00,776796.69
3,2023-08-04,19523,83.0,33200.0,42.57,0.0,0.00,810039.25
4,2023-08-05,19523,0.0,0.0,44.39,0.0,0.00,810083.64
5,2023-08-06,19523,0.0,0.0,44.39,0.0,0.00,810128.03
6,2023-08-07,19525,2.0,800.0,44.39,0.0,0.00,810972.42
7,2023-08-08,19136,-389.0,-155600.0,44.44,0.0,0.00,655416.86
8,2023-08-09,19236,100.0,40000.0,35.91,0.0,0.00,695452.77
9,2023-08-10,19248,12.0,4800.0,38.11,0.0,0.00,700290.88


In [3]:
import math
import pandas as pd
from datetime import datetime

R = 0.02
N_CONTRACTS = 8
IM = 101_944.0
FEE_PER_SIDE = 110.54
SETTLEMENT_FEE_PER_CONTRACT = 10.0

q1 = q1_table.copy()
q1["Date"] = pd.to_datetime(q1["Date"])

t0 = q1["Date"].iloc[0]
tT = q1["Date"].iloc[-1]
days = (tT - t0).days


open_fee  = N_CONTRACTS * FEE_PER_SIDE
close_fee = N_CONTRACTS * FEE_PER_SIDE
settle_fee = N_CONTRACTS * SETTLEMENT_FEE_PER_CONTRACT

# Present Value of Invested cash at t0 (初始保证金 + 折现后的追加保证金 + 开仓手续费)
pv_invested = N_CONTRACTS * IM + open_fee       # 开仓手续费发生在t0，不贴现
for d, dep in zip(q1["Date"], q1["Deposit"]):
    dep = float(dep)
    if dep != 0.0:
        dt = (d - t0).days
        pv_dep = dep * math.exp(-R * dt / 365.0)   # 按2%连续贴现回t0
        pv_invested += pv_dep

# 期末净额：账户余额 - 平仓手续费 - 结算费（不再扣开仓手续费）
final_balance = float(q1["Balance"].iloc[-1])
final_net = final_balance - (close_fee + settle_fee)

# HPR 和 年化
hpr = (final_net - pv_invested) / pv_invested
annualized = (1 + hpr) ** (365.0 / days) - 1

print(f"PV(Invested) @ 2% cont. (incl. open fee): {pv_invested:,.2f} HKD")
print(f"Final Balance: {final_balance:,.2f} HKD")
print(f"Fees at T (close + settlement): {(close_fee + settle_fee):,.2f} HKD")
print(f"Final Net (after fees at T): {final_net:,.2f} HKD")
print(f"Holding Period Return (HPR): {hpr*100:.2f}%")
print(f"Annualized Return: {annualized*100:.2f}%")


PV(Invested) @ 2% cont. (incl. open fee): 1,684,394.22 HKD
Final Balance: 934,158.18 HKD
Fees at T (close + settlement): 964.32 HKD
Final Net (after fees at T): 933,193.86 HKD
Holding Period Return (HPR): -44.60%
Annualized Return: -99.70%


In [5]:
# spot HSI start and end
hsi_start = data["HSI"].iloc[0]
hsi_end   = data["HSI"].iloc[-1]

# 离散复利
hsi_hpr = (hsi_end - hsi_start) / hsi_start
hsi_annualized = (1 + hsi_hpr) ** (365 / days) - 1

print("HSI Start:", hsi_start)
print("HSI End:", hsi_end)
print("HSI Holding Period Return:", round(hsi_hpr*100, 2), "%")
print("HSI Annualized Return:", round(hsi_annualized*100, 2), "%")

HSI Start: 20011
HSI End: 18202
HSI Holding Period Return: -9.04 %
HSI Annualized Return: -60.73 %
